In [1]:
import openml
import tpot2
import sklearn.metrics
import sklearn
from sklearn.metrics import (roc_auc_score, log_loss)
import traceback
import dill as pickle
import os
import time
import numpy as np
import sklearn.model_selection

def load_task(task_id, preprocess=True):

    cached_data_path = f"data/{task_id}_{preprocess}.pkl"
    print(cached_data_path)
    if os.path.exists(cached_data_path):
        d = pickle.load(open(cached_data_path, "rb"))
        X_train, y_train, X_test, y_test = d['X_train'], d['y_train'], d['X_test'], d['y_test']
    else:
        task = openml.tasks.get_task(task_id)


        X, y = task.get_X_and_y(dataset_format="dataframe")
        train_indices, test_indices = task.get_train_test_split_indices()
        X_train = X.iloc[train_indices]
        y_train = y.iloc[train_indices]
        X_test = X.iloc[test_indices]
        y_test = y.iloc[test_indices]

        if preprocess:
            preprocessing_pipeline = sklearn.pipeline.make_pipeline(tpot2.builtin_modules.ColumnSimpleImputer("categorical", strategy='most_frequent'), tpot2.builtin_modules.ColumnSimpleImputer("numeric", strategy='mean'), tpot2.builtin_modules.ColumnOneHotEncoder("categorical", min_frequency=0.001, handle_unknown="ignore"))
            X_train = preprocessing_pipeline.fit_transform(X_train)
            X_test = preprocessing_pipeline.transform(X_test)


            le = sklearn.preprocessing.LabelEncoder()
            y_train = le.fit_transform(y_train)
            y_test = le.transform(y_test)

            X_train = X_train.to_numpy()
            X_test = X_test.to_numpy()

            if task_id == 168795: #this task does not have enough instances of two classes for 10 fold CV. This function samples the data to make sure we have at least 10 instances of each class
                indices = [28535, 28535, 24187, 18736,  2781]
                y_train = np.append(y_train, y_train[indices])
                X_train = np.append(X_train, X_train[indices], axis=0)

            d = {"X_train": X_train, "y_train": y_train, "X_test": X_test, "y_test": y_test}
            if not os.path.exists("data"):
                os.makedirs("data")
            with open(cached_data_path, "wb") as f:
                pickle.dump(d, f)

    return X_train, y_train, X_test, y_test

# Australian (classification)

In [2]:
X_train, y_train, X_test, y_test = load_task(167104, preprocess=True)


data/167104_True.pkl


In [3]:
est = tpot2.TPOTEstimator(generations=7, population_size=15, cv=5, 
                                    random_state=42, verbose=2, classification=True, scorers=['roc_auc_ovr',tpot2.objectives.complexity_scorer], scorers_weights=[1,-1])
est.fit(X_train, y_train)

scorer = sklearn.metrics.get_scorer('roc_auc_ovo')
print(scorer(est, X_test, y_test))

Generation: 100%|█████████████████████████████████| 7/7 [01:03<00:00,  9.06s/it]


0.9150763358778625


In [4]:
est.pareto_front

,roc_auc_score,complexity_scorer,Parents,Variation_Function,Individual,Generation,Submitted Timestamp,Completed Timestamp,Eval Error,Pareto_Front,Instance
56,0.937486,6.0,"(29, 29)",ind_mutate,<GraphIdnividual 0,3.0,1.725343e+09,1.725343e+09,None,1.0,"[('BernoulliNB_1', 'RFE_ExtraTreesClassifier_1..."
73,0.943420,2346.0,"(2, 2)",ind_mutate,<GraphIdnividual 0,4.0,1.725343e+09,1.725343e+09,None,1.0,['XGBClassifier_1']
85,0.933883,2.0,"(43, 19)",ind_crossover,<GraphIdnividual 0,5.0,1.725343e+09,1.725343e+09,None,1.0,"[('MultinomialNB_1', 'MaxAbsScaler_1')]"
86,0.943553,2964.0,"(52, 52)",ind_mutate,<GraphIdnividual 0,5.0,1.725343e+09,1.725343e+09,None,1.0,['XGBClassifier_1']
92,0.735111,1.0,"(76, 76)",ind_mutate,<GraphIdnividual 0,6.0,1.725343e+09,1.725343e+09,None,1.0,['MultinomialNB_1']
96,0.945411,3180.0,"(2, 2)",ind_mutate,<GraphIdnividual 0,6.0,1.725343e+09,1.725343e+09,None,1.0,['XGBClassifier_1']


In [5]:
pf = est.pareto_front

In [6]:
# from sklearn.utils.validation import check_is_fitted
# est = pf.iloc[4]['Individual'].export_pipeline()
# est.fit(X_train, y_train)
# print(est)
# print(type(est))
# check_is_fitted(est)

# results = est.predict(X_test)
# roc_auc = roc_auc_score(y_test, results)
# print(f"ROC_AUC: {roc_auc * 100:.2f}%")


In [7]:
# test_clf = pf.iloc[4][10]

# test_clf.fit(X_train, y_train)

# results = test_clf.predict(X_test)

# roc_auc = roc_auc_score(y_test, results)
# print(f"ROC_AUC: {roc_auc * 100:.2f}%")


In [9]:
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score




estimators = []

for i in range(len(pf)):
    # print(pf.iloc[i, 0])
    fitted_pipeline = pf.iloc[i, 10].fit(X_train, y_train)
    #print(roc_auc_score(y_test, fitted_pipeline.predict(X_test)))
    print(accuracy_score(y_test, fitted_pipeline.predict(X_test)))

    
    fitted_pipeline_tuple = ((str(i), fitted_pipeline))
    estimators.append(fitted_pipeline_tuple)



stacking_classifier = StackingClassifier(estimators=estimators, 
                                         final_estimator=VotingClassifier(estimators=estimators, voting='hard'), 
                                         cv="prefit",
                                         passthrough=True)
stacking_classifier

stacking_classifier.fit(X_train, y_train)

0.8502202643171806
0.8458149779735683
0.8502202643171806
0.8237885462555066
0.7577092511013216
0.8370044052863436


StackingClassifier(cv='prefit',
                   estimators=[('0',
                                GraphPipeline(graph=<networkx.classes.digraph.DiGraph object at 0x143c09610>)),
                               ('1',
                                GraphPipeline(graph=<networkx.classes.digraph.DiGraph object at 0x143bbcb90>)),
                               ('2',
                                GraphPipeline(graph=<networkx.classes.digraph.DiGraph object at 0x143bbf740>)),
                               ('3',
                                GraphPipeline(graph=<networkx.classes.digraph.DiGraph object...
                                                                 GraphPipeline(graph=<networkx.classes.digraph.DiGraph object at 0x143bbcb90>)),
                                                                ('2',
                                                                 GraphPipeline(graph=<networkx.classes.digraph.DiGraph object at 0x143bbf740>)),
                                                                ('3',
                                                                 GraphPipeline(graph=<networkx.classes.digraph.DiGraph object at 0x143bbd160>)),
                                                                ('4',
                                                                 GraphPipeline(graph=<networkx.classes.digraph.DiGraph object at 0x143bbc0b0>)),
                                                                ('5',
                                                                 GraphPipeline(graph=<networkx.classes.digraph.DiGraph object at 0x143bbfd40>))]),
                   passthrough=True)

In [10]:


results = stacking_classifier.predict(X_test)
accuracy = accuracy_score(y_test, results)
#roc_auc = roc_auc_score(y_test, results)
print(f"Accuracy: {accuracy * 100:.4f}%")
#print(f"ROC_AUC: {roc_auc * 100:.4f}%")

Accuracy: 85.9031%


# Vehicle (classification)

In [11]:
X_train, y_train, X_test, y_test = load_task(167168, preprocess=True)

data/167168_True.pkl


In [12]:
est = tpot2.TPOTEstimator(generations=5, population_size=10, cv=5, 
                          random_state=42, verbose=2, classification=True, scorers=['roc_auc_ovr',tpot2.objectives.complexity_scorer], 
                          scorers_weights=[1,-1])
est.fit(X_train, y_train)

scorer = sklearn.metrics.get_scorer('roc_auc_ovo')
print(scorer(est, X_test, y_test))

Generation: 100%|█████████████████████████████████| 5/5 [00:35<00:00,  7.12s/it]


0.9431420585289919


In [13]:
pf = est.pareto_front

In [14]:
pf

,roc_auc_score,complexity_scorer,Parents,Variation_Function,Individual,Generation,Submitted Timestamp,Completed Timestamp,Eval Error,Pareto_Front,Instance
5,0.863892,198.0,NaN,NaN,<GraphIdnividual 0,0.0,1.725343e+09,1.725343e+09,None,1.0,"[('DecisionTreeClassifier_1', 'ZeroCount_1'), ..."
7,0.906244,13554.0,NaN,NaN,<GraphIdnividual 0,0.0,1.725343e+09,1.725343e+09,None,1.0,['XGBClassifier_1']
8,0.914528,19708.0,NaN,NaN,<GraphIdnividual 0,0.0,1.725343e+09,1.725343e+09,None,1.0,['RandomForestClassifier_1']
11,0.917348,24863.0,"(3, 3)",ind_mutate,<GraphIdnividual 0,1.0,1.725343e+09,1.725343e+09,None,1.0,"[('RandomForestClassifier_1', 'StandardScaler_..."
13,0.912651,19663.0,"(8, 5)",ind_crossover,<GraphIdnividual 0,1.0,1.725343e+09,1.725343e+09,None,1.0,"[('RandomForestClassifier_1', 'ZeroCount_1')]"
17,0.496933,27.0,"(9, 9)",ind_mutate,<GraphIdnividual 0,1.0,1.725343e+09,1.725343e+09,None,1.0,"[('DecisionTreeClassifier_1', 'RobustScaler_1'..."
30,0.932951,32179.0,"(28, 28)",ind_mutate,<GraphIdnividual 0,3.0,1.725343e+09,1.725343e+09,None,1.0,"[('RandomForestClassifier_1', 'Normalizer_1'),..."
32,0.901994,10958.0,"(8, 8)",ind_mutate,<GraphIdnividual 0,3.0,1.725343e+09,1.725343e+09,None,1.0,['RandomForestClassifier_1']
36,0.932340,31769.0,"(28, 11)",ind_crossover,<GraphIdnividual 0,3.0,1.725343e+09,1.725343e+09,None,1.0,"[('RandomForestClassifier_1', 'StandardScaler_..."
44,0.860562,188.0,"(5, 5)",ind_mutate,<GraphIdnividual 0,4.0,1.725343e+09,1.725343e+09,None,1.0,"[('DecisionTreeClassifier_1', 'ZeroCount_1'), ..."


In [15]:
estimators = []

for i in range(len(pf)):
    fitted_pipeline = pf.iloc[i, 10].fit(X_train, y_train)
    
    #print(roc_auc_score(y_test, fitted_pipeline.predict_proba(X_test), multi_class='ovr'))
    print(accuracy_score(y_test, fitted_pipeline.predict(X_test)))
    
    fitted_pipeline_tuple = ((str(i), fitted_pipeline))
    estimators.append(fitted_pipeline_tuple)



stacking_classifier = StackingClassifier(estimators=estimators, 
                                         final_estimator=VotingClassifier(estimators=estimators, voting='soft'), 
                                         cv="prefit",
                                         passthrough=False)
stacking_classifier

stacking_classifier.fit(X_train, y_train) 

/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_selection/_base.py:116: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_selection/_base.py:116: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  warnings.warn(


0.6021505376344086
0.6953405017921147
0.7096774193548387
0.7240143369175627
0.7060931899641577
0.21863799283154123
0.7526881720430108
0.6881720430107527
0.7491039426523297
0.6272401433691757


/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_selection/_base.py:116: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_selection/_base.py:116: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  warnings.warn(


0.7204301075268817


/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_selection/_base.py:116: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_selection/_base.py:116: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_selection/_base.py:116: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_selection/_base.py:116: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  warnings.warn(


StackingClassifier(cv='prefit',
                   estimators=[('0',
                                GraphPipeline(graph=<networkx.classes.digraph.DiGraph object at 0x1450e0200>)),
                               ('1',
                                GraphPipeline(graph=<networkx.classes.digraph.DiGraph object at 0x145142270>)),
                               ('2',
                                GraphPipeline(graph=<networkx.classes.digraph.DiGraph object at 0x1451fc380>)),
                               ('3',
                                GraphPipeline(graph=<networkx.classes.digraph.DiGraph object...
                                                                 GraphPipeline(graph=<networkx.classes.digraph.DiGraph object at 0x14529d670>)),
                                                                ('7',
                                                                 GraphPipeline(graph=<networkx.classes.digraph.DiGraph object at 0x14529da00>)),
                                                                ('8',
                                                                 GraphPipeline(graph=<networkx.classes.digraph.DiGraph object at 0x14529e1b0>)),
                                                                ('9',
                                                                 GraphPipeline(graph=<networkx.classes.digraph.DiGraph object at 0x14529c3b0>)),
                                                                ('10',
                                                                 GraphPipeline(graph=<networkx.classes.digraph.DiGraph object at 0x14529ea50>))],
                                                    voting='soft'))

In [16]:
# results = stacking_classifier.predict_proba(X_test)

# roc_auc = roc_auc_score(y_test, results, multi_class='ovr')
# print(f"ROC_AUC: {roc_auc * 100:.4f}%")

accuracy = accuracy_score(y_test, stacking_classifier.predict(X_test))
print(f"Accuracy: {accuracy * 100:.4f}%")

Accuracy: 75.6272%


/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_selection/_base.py:116: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_selection/_base.py:116: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_selection/_base.py:116: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_selection/_base.py:116: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  warnings.warn(


In [17]:
def create_stacking_clf(pareto_front):
    estimators = []
    for i in range(len(pareto_front)):
        fitted_pipeline = pareto_front.iloc[i, 10].fit(X_train, y_train)
        # print(roc_auc_score(y_test, fitted_pipeline.predict_proba(X_test), multi_class='ovr'))
        print(accuracy_score(y_test, fitted_pipeline.predict(X_test)))
        fitted_pipeline_tuple = ((str(i), fitted_pipeline))
        estimators.append(fitted_pipeline_tuple)

    ensemble_model_params = {
        'passthrough': [True, False],  
        'final_estimator': [
            VotingClassifier(estimators=estimators, voting='soft'), 
            VotingClassifier(estimators=estimators, voting='hard'),
            #LogisticRegression()
        ]
    }

    ensemble_grid_search = GridSearchCV(estimator=StackingClassifier(estimators=estimators, cv="prefit"), 
                                        param_grid=ensemble_model_params, 
                                        cv=5)


    ensemble_grid_search.fit(X_train, y_train)
    print(type(ensemble_grid_search.best_params_))

    
    return ensemble_grid_search

**testing with grid search**

In [18]:
X_train, y_train, X_test, y_test = load_task(167168, preprocess=True)

est = tpot2.TPOTEstimator(generations=5, population_size=10, cv=5, 
                          random_state=42, verbose=2, classification=True, scorers=['roc_auc_ovr',tpot2.objectives.complexity_scorer], 
                          scorers_weights=[1,-1])
est.fit(X_train, y_train)

fitted_ensemble = create_stacking_clf(est.pareto_front)

accuracy = accuracy_score(y_test, fitted_ensemble.predict(X_test))
print("ensemble score:")
print(f"Accuracy: {accuracy * 100:.4f}%")

data/167168_True.pkl


Generation: 100%|█████████████████████████████████| 5/5 [00:36<00:00,  7.29s/it]
/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_selection/_base.py:116: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_selection/_base.py:116: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  warnings.warn(


0.6021505376344086
0.6953405017921147
0.7096774193548387
0.7240143369175627
0.7060931899641577
0.21863799283154123
0.7526881720430108
0.6881720430107527
0.7491039426523297
0.6272401433691757


/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_selection/_base.py:116: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_selection/_base.py:116: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  warnings.warn(


0.7204301075268817


/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_selection/_base.py:116: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_selection/_base.py:116: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_selection/_base.py:116: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_selection/_base.py:116: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_selection/_base.py:116: UserWarning: No features were selected: either the data is too noisy or the selection test too stric

<class 'dict'>
ensemble score:
Accuracy: 75.9857%


/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_selection/_base.py:116: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_selection/_base.py:116: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_selection/_base.py:116: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  warnings.warn(


In [19]:
print(fitted_ensemble.best_params_)

{'final_estimator': VotingClassifier(estimators=[('0',
                              GraphPipeline(graph=<networkx.classes.digraph.DiGraph object at 0x14561ad50>)),
                             ('1',
                              GraphPipeline(graph=<networkx.classes.digraph.DiGraph object at 0x145707e30>)),
                             ('2',
                              GraphPipeline(graph=<networkx.classes.digraph.DiGraph object at 0x145704c50>)),
                             ('3',
                              GraphPipeline(graph=<networkx.classes.digraph.DiGraph object at 0x145704140>...
                              GraphPipeline(graph=<networkx.classes.digraph.DiGraph object at 0x145770380>)),
                             ('7',
                              GraphPipeline(graph=<networkx.classes.digraph.DiGraph object at 0x1457701a0>)),
                             ('8',
                              GraphPipeline(graph=<networkx.classes.digraph.DiGraph object at 0x145770c80>)),


In [26]:
X_train, y_train, X_test, y_test = load_task(167104, preprocess=True)

est = tpot2.TPOTEstimator(generations=5, population_size=10, cv=5, 
                          random_state=48, verbose=2, classification=True, scorers=['roc_auc_ovr',tpot2.objectives.complexity_scorer], 
                          scorers_weights=[1,-1])
est.fit(X_train, y_train)

fitted_ensemble = create_stacking_clf(est.pareto_front)

accuracy = accuracy_score(y_test, fitted_ensemble.predict(X_test))
print("ensemble score:")
print(f"Accuracy: {accuracy * 100:.4f}%")


data/167104_True.pkl


Generation: 100%|█████████████████████████████████| 5/5 [00:14<00:00,  3.00s/it]


0.8325991189427313
0.8237885462555066
0.8325991189427313
0.8502202643171806
0.7577092511013216
0.8325991189427313
0.8237885462555066
0.8193832599118943
<class 'dict'>
ensemble score:
Accuracy: 83.2599%


In [21]:
print(pd.DataFrame(fitted_ensemble.cv_results_))

NameError: name 'pd' is not defined

In [ ]:
print(fitted_ensemble.best_estimator_)